In [1]:
import csv
import re
import requests
import time
from bs4 import BeautifulSoup

# scrapes the title 
def get_title():
	d = soup.find_all("h1", "branded-page-header-title")
	for i in d:
		title = i.text.strip().replace('\n',' ').replace(',','').encode("utf-8") 
		f.write(title+',')
		print('\t%s') % (title)

# scrapes the subscriber count
def get_subs():
	b = soup.find_all("span", "about-stat")
	for i in b:
		try:			
			value = i.b.text.strip().replace(',','')					
			if len(b) == 3:
				f.write(value+',')
				print('\t%s') %(value)
			elif len(b) == 2:
				f.write('null,'+ value + ',')
				print('\tsubs = null\n\t%s') %(value)
			else:
				f.write('null,null,')
				print('\tsubs = null\nviews = null')
		except AttributeError:
			pass

# scrapes the description
def get_description():
	c = soup.find_all("div", "about-description")
	if c:
		for i in c:
			description = i.text.strip().replace('\n',' ').replace(',','').encode("utf-8")		
			f.write(description+',')
			print('\t%s') % (description)
			

			regex = re.compile(("([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
						        "{|}~-]+)*(@|\sat\s|\[at\])(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
						        "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)"))
			
			email = re.search(regex, description)
			if email:
				if not email.group(0).startswith('//'):
					print('\tEmail = ' + email.group())
					f.write(email.group(0)+',')
			else:
				print('\tEmail = null')
				f.write('null,')
	else:
		print('\tDescription = null\n\tEmail = null')
		f.write('null,null,')

		

# scrapes all the external links 
def get_links():
	a = soup.find_all("a", "about-channel-link ") # trailing space is required.
	for i in a:
		url = i.get('href')
		f.write(url+',')
		print('\t%s') % (url)

# scrapes the related channels
def get_related():
	s = soup.find_all("h3", "yt-lockup-title")
	for i in s:
		t = i.find_all(href=re.compile("user"))
		for i in t:
			url = 'https://www.youtube.com'+i.get('href')
			related.write(url+'\n')
			#print('\t\t%s,%s') % (i.text, url)	


def add_protocol(url):
	if (url.startswith("http://") or url.startswith("https://")): 
		return url
	else:
		url = 'http://' + url
		return url

f = open("email-scrape-data.csv", "w+") 
s = ['dkohreidze.com/jquery', 'http://rivercitiescustomconcepts.com', 'www.paintingsa.com/', 'http://paradisegardentx.com/', 'http://www.propaintingandstaining.com', 'http://www.handhvinylfencing.com', 'http://www.catrinasranchinteriors.com/'] # enumerate all keywords here
siteCount = 0
emailCount = 0

for site in s:
	siteCount += 1

	# ping and retrieve home page HTML 
	r = requests.get(add_protocol(site))

	# create Soup object from HTML 
	soup = BeautifulSoup(r.text)

	print('\nScraping %s') %(add_protocol(site))

	mailto = soup.find(href=re.compile("mailto"))

	if mailto:
		email = re.sub('mailto:', '', mailto.get('href').lower()) 
		f.write(email + ',' + site + '\n')
		print email + ',' + site
		emailCount += 1
	else: # move on to contact page
		print('checking contact page for %s') %site
		 
		contact = soup.find(href=re.compile("contact"))
		
		if contact:
			if str(contact).startswith('/'):
				contactURL = str(site + contact.get('href').lower())
				print contactURL
			elif str(contact).startswith('http'):	
				contactURL = contact.get('href').lower()
				print contactURL
			else:
				contactURL = str(site + '/' + contact.get('href').lower())
				print contactURL

			r = requests.get(add_protocol(contactURL))
			soup = BeautifulSoup(r.text)
			
			print('\nScraping %s') %(contactURL)
		
			mailto = soup.find(href=re.compile("mailto"))

			if mailto:
				email = re.sub('mailto:', '', mailto.get('href').lower()) 
				f.write(email + ',' + site + '\n')
				print email + ',' + site
				emailCount += 1
			else:
				print('Cannot find email.')
		else:
			pass

successRate = float(emailCount) / float(siteCount) * 100.0
print('\nSites scraped = %d\nEmails captured = %d\nSuccess rate = %d%%') %(siteCount, emailCount, successRate)
f.close()

/home/miti/.local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))



Scraping http://dkohreidze.com/jquery
checking contact page for dkohreidze.com/jquery
dkohreidze.com/jquery/http://dkohreidze.com/#contact

Scraping dkohreidze.com/jquery/http://dkohreidze.com/#contact
Cannot find email.

Scraping http://rivercitiescustomconcepts.com
williamw@rivercitiescustomconcepts.com,http://rivercitiescustomconcepts.com

Scraping http://www.paintingsa.com/
powellpainting@gmail.com,www.paintingsa.com/

Scraping http://paradisegardentx.com/
checking contact page for http://paradisegardentx.com/

Scraping http://www.propaintingandstaining.com
jimmy@propaintingandstaining.com,http://www.propaintingandstaining.com

Scraping http://www.handhvinylfencing.com
eric@handhvinylfencing.com,http://www.handhvinylfencing.com

Scraping http://www.catrinasranchinteriors.com/
checking contact page for http://www.catrinasranchinteriors.com/
http://www.catrinasranchinteriors.com//contactus.html

Scraping http://www.catrinasranchinteriors.com//contactus.html
catrina@catrinasranchinte